In [1]:
import pandas as pd
import re

In [11]:
# Example of data to extract
data = ['98 LH EATXIIIA WITHOUT AUTOSTICK\nLH', '95 BC EATXIIIA WITHOUT AUTOSTICK', '2003 EATXIIIA WITHOUT LA AUTOSTICK']

In [12]:
# Convert to df 
df = pd.DataFrame(data)

In [13]:
# Print the df
df

,0
0,98 LH EATXIIIA WITHOUT AUTOSTICK\nLH
1,95 BC EATXIIIA WITHOUT AUTOSTICK
2,2003 EATXIIIA WITHOUT LA AUTOSTICK


In [14]:
# Create a pattern to identify the body code
pattern = r'\s+([A-Z][A-Z0-9])\s+'

In [16]:
# List to store the positions and the body codes
positions = []

# Iterate under column 0 to get the text
for idx, text in df[0].items():
    # Iterate under the whole text and use the regex pattern to capture the body code 
    for match in re.finditer(pattern, text):
        # Append body code and the start position of the body code
        positions.append({
            'index': idx,
            'BC': match.group(1),
            'start_pos': match.start(1),
        })

In [17]:
# Print the list
positions

[{'index': 0, 'BC': 'LH', 'start_pos': 3},
 {'index': 1, 'BC': 'BC', 'start_pos': 3},
 {'index': 2, 'BC': 'LA', 'start_pos': 22}]

In [18]:
positions[1]

{'index': 1, 'BC': 'BC', 'start_pos': 3}

In [19]:
# Convert the list to df
df_positions = pd.DataFrame(positions)

In [20]:
# Print the df
df_positions

,index,BC,start_pos
0,0,LH,3
1,1,BC,3
2,2,LA,22


In [21]:
# Concate the df with the body code and position with the original df
# axis=1 is used to indicate the df will be concatenated horizontally, that is, with the columns side by side 
df_concatenated = pd.concat([df, df_positions], axis=1)

In [22]:
# Print the df concatenated
df_concatenated

,0,index,BC,start_pos
0,98 LH EATXIIIA WITHOUT AUTOSTICK\nLH,0,LH,3
1,95 BC EATXIIIA WITHOUT AUTOSTICK,1,BC,3
2,2003 EATXIIIA WITHOUT LA AUTOSTICK,2,LA,22


#### JOINING MULTIPLE BODY CODES AT THE SAME ROW

In [23]:
body_code1 = 'LA'

In [24]:
body_code2 = 'BC'

In [25]:
# Join the above strings
body_code = ', '.join([body_code1, body_code2])

In [26]:
# Print the strings joined
body_code

'LA, BC'

In [27]:
# Data example
data = {
    'index': [14, 229, 229, 229, 230, 230],
    'BC': ['LA', 'BC', 'A3', 'B1', 'B2', 'C1']
    }

In [28]:
# Convert to df
df = pd.DataFrame(data)

In [29]:
# Print the df
df

,index,BC
0,14,LA
1,229,BC
2,229,A3
3,229,B1
4,230,B2
5,230,C1


In [30]:
# Print the df after checked marked
df

,index,BC
0,14,LA
1,229,BC
2,229,A3
3,229,B1
4,230,B2
5,230,C1


In [31]:
# groupby method group data from the same column to one and agg method indicates the next data should be grouped from
df.groupby(['index']).agg({'BC': list})

,BC
index,
14,[LA]
229,"[BC, A3, B1]"
230,"[B2, C1]"


#### Compare 2 lists and find matches 

In [32]:
# Create a list with examples of bcs
bc_list1 = ['A1', 'BC', 'B2', 'B1', '11']

In [33]:
# Create a second list 
bc_list2 = ['BC', 'A1', 'B1', 'BA', '20']

In [34]:
# Create a list with items that are in both lists
match_bc_list = []

# Iterate under the first list
for bc in bc_list1:
    # Iterate under the 2nd lit
    for bc1 in bc_list2:
        # Condition to confirm whether the bcs match
        if bc == bc1:
            # Append the matched bcs
            match_bc_list.append(bc)

In [35]:
# Print the match list
match_bc_list

['A1', 'BC', 'B1']

#### Test merge method to merge 2 different dfs with different sizes

In [36]:
# Data example 1
data1 = {
    'index': [0, 1, 2, 3, 4],
    'Year': ['A1', pd.NA, 'B2', pd.NA, '11']
    }

In [37]:
# Data example 2
data2 = {
    'index': [1, 3],
    'Year': ['BC, CB', 'B1']
    }

In [38]:
# Convert both data examples to df
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

In [39]:
# Merge both dfs
# on - is the column reference or the column which matches in both dfs
# outer: use union of keys from both frames
dfs_merged = df1.merge(df2, how='outer', on='index')

In [40]:
dfs_merged

,index,Year_x,Year_y
0,0,A1,NaN
1,1,<NA>,"BC, CB"
2,2,B2,NaN
3,3,<NA>,B1
4,4,11,NaN


In [41]:
dfs_merged['Year_x'] = dfs_merged['Year_x'].fillna(dfs_merged['Year_y'])

In [42]:
dfs_merged

,index,Year_x,Year_y
0,0,A1,NaN
1,1,"BC, CB","BC, CB"
2,2,B2,NaN
3,3,B1,B1
4,4,11,NaN


### TEST A REMOVE DUPES AND COMBINE DOUBLE CODES METHOD

In [108]:
# Create a df for testing the method
data = pd.DataFrame( 
    data=[['BC, B1, B1'], ['W2, WK, WK'], ['']],
    index=[2002, 2011, 2012],
    columns=['bc']
    )

In [109]:
# Print the df
data

,bc
2002,"BC, B1, B1"
2011,"W2, WK, WK"
2012,


In [110]:
# Body code pattern
bc_pattern = r'[A-Z0-9]{2}'

In [111]:
# Create a list with unique body codes
unique_bc_list = []

# Iterate under column bc to get the string with the bcs
for bc in data['bc']:
    # Method findall find the matches based on the pattern
    bc_matches = re.findall(bc_pattern, bc)
    if bc != '':
        bc_unique = list(set(bc_matches))
        bc_out_of_list = ', '.join(bc_unique)
        unique_bc_list.append(bc_out_of_list)
    else:
        unique_bc_list.append(bc)

In [112]:
# Print the list
unique_bc_list

['B1, BC', 'WK, W2', '']

In [113]:
# Update the df after removing the duplicated bcs
data['bc'] = unique_bc_list  

In [114]:
# Print the updated df
data

,bc
2002,"B1, BC"
2011,"WK, W2"
2012,
